### Teoría

- Esta parte será el 40% de la nota final del bloque de Aprendizaje por Refuerzo.

Define brevemente qué es el aprendizaje por refuerzo. ¿Qué diferencias hay entre aprendizaje supervisado, no supervisado y por refuerzo?

Define con tus palabras los conceptos de Entorno, Agente, Recompensa, Estado y Observación.

Dependiendo del algoritmo de aprendizaje por refuerzo que se use, ¿qué clasificaciones podemos encontrar? Coméntalas brevemente.

Lista tres diferencias entre los algoritmos de DQN y Policy Gradient

### Práctica

- Esta parte será el 60% de la nota final del bloque de Aprendizaje por Refuerzo.

Algunas consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, una solución óptima será alcanzar una media de recompensa por encima de 16 puntos. Para medir si hemos conseguido llegar a la solución óptima, la media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este bloque práctico consta de tres partes:

   1) Implementar la red neuronal que se usará en la solución
    
   2) Seleccionar los hiperparámetros adecuados para las distintas piezas de la solución DQN
    
   3) Justificar la respuesta en relación a los resultados obtenidos

IMPORTANTE:

- Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.

- Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).

- Si usáis Google Colab, recordad usar las versiones de Tensorflow==1.13.1, Keras==2.2.4 y keras-rl==0.4.2

In [2]:
# Uncomment this line for installing keras-rl on Google collaboratory
!pip install keras-rl

In [5]:
%pip install gym[atari]

Note: you may need to restart the kernel to use updated packages.


In [3]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

Using TensorFlow backend.
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [6]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [7]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1) Implementación de la red neuronal

In [8]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=input_shape))

# TODO

model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

W0415 21:08:48.764111 10936 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0415 21:08:48.811576 10936 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0415 21:08:48.836408 10936 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
dense_1 (Dense)              (None, 84, 84, 6)         30        
_________________________________________________________________
activation_1 (Activation)    (None, 84, 84, 6)         0         
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
None


2) Selección de hiperparámetros para la solución DQN

In [12]:
# TODO - Select the parameters for the memory
memory = SequentialMemory(limit=10, window_length=5)
processor = AtariProcessor()

In [18]:
# TODO - Select the parameters for the policy
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=5, value_min=3, value_test=3, nb_steps=5)

In [21]:
# TODO - Select the parameters for the Agent and the Optimizer
dqn = DQNAgent(model, nb_actions=5, memory=memory)
dqn.compile(Adam(), metrics=['mae'])

ValueError: Model output "Tensor("activation_1/Identity:0", shape=(?, 84, 84, 6), dtype=float32)" has invalid shape. DQN expects a model that has one dimension for each action, in this case 5.

In [ ]:
# Training part
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]

# TODO - Select the parameters for the method "fit"
dqn.fit()

dqn.save_weights(weights_filename, overwrite=True)

In [ ]:
# Testing part to calculate the mean reward
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

3) Justificación de los parámetros seleccionados y de los resultados obtenidos